# PROCESSING ANNOTATED ENTITIES AND TRIPLES
For evaluating 10 folds

## Entities of 10 folds

In [2]:
import pandas as pd
# from transformers import AutoTokenizer, AutoModel
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:

def get_ent2types (df):
    All = {}
    All['disease'], All ['species'], All['drug'], All['gene'], All ['symptom'], All ['treat-procedure'] = [],[],[],[],[],[]
    for o, s, r in zip (df['object'], df['subject'], df['relation']):
        keys = r.split('_')
        All[keys[0]].append(s)
        All[keys[1]].append(o)
    print(All.keys())
    return All

In [5]:
n_subsets = 10
All = {}
All['subset'], All['disease'], All ['species'], All['drug'], All['gene'], All ['symptom'], All ['treat-procedure'] = [],[],[],[],[],[],[]

for i in range(1, n_subsets+1):
    with open (r"C:\Users\huyen\OneDrive\Documents\GitHub\CORD-19-KG\Data\all-final-cleaned-triple3-10sets\subset_%s.csv"%i, 'r', encoding = 'utf-8') as f:
        data = pd.read_csv(f)
    data.head()
    data['relation'] = data['relation'].apply(lambda x: x.replace('treat_', 'treat-'))

    entities_2types = get_ent2types (data)
#     print('\n------------subset_%s ----------------'%i)
    All['subset'].append(i)
    for k in entities_2types.keys():
#         print('%s: %s' %(k, len(entities_2types[k])))
        All[k].append(len(entities_2types[k]))
        
All

dict_keys(['disease', 'species', 'drug', 'gene', 'symptom', 'treat-procedure'])
dict_keys(['disease', 'species', 'drug', 'gene', 'symptom', 'treat-procedure'])
dict_keys(['disease', 'species', 'drug', 'gene', 'symptom', 'treat-procedure'])
dict_keys(['disease', 'species', 'drug', 'gene', 'symptom', 'treat-procedure'])
dict_keys(['disease', 'species', 'drug', 'gene', 'symptom', 'treat-procedure'])
dict_keys(['disease', 'species', 'drug', 'gene', 'symptom', 'treat-procedure'])
dict_keys(['disease', 'species', 'drug', 'gene', 'symptom', 'treat-procedure'])
dict_keys(['disease', 'species', 'drug', 'gene', 'symptom', 'treat-procedure'])
dict_keys(['disease', 'species', 'drug', 'gene', 'symptom', 'treat-procedure'])
dict_keys(['disease', 'species', 'drug', 'gene', 'symptom', 'treat-procedure'])


{'subset': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 'disease': [8650,
  16699,
  25962,
  33735,
  41371,
  48844,
  56418,
  64173,
  71629,
  79212],
 'species': [827, 1643, 2579, 3912, 5267, 6592, 7897, 9136, 10283, 11436],
 'drug': [1028, 2265, 3342, 4750, 6362, 8050, 9617, 11249, 12817, 14472],
 'gene': [5195, 10835, 15487, 19730, 23847, 27922, 32345, 36649, 41215, 45616],
 'symptom': [197, 376, 605, 1536, 2300, 3156, 3941, 4674, 5400, 6148],
 'treat-procedure': [1867,
  3710,
  5317,
  7393,
  9673,
  12018,
  14126,
  16225,
  18524,
  20746]}

In [6]:
df = pd.DataFrame.from_dict(All)
df = df[['subset', 'symptom', 'species', 'drug', 'treat-procedure', 'gene', 'disease' ]]
df
# with open(r'.\tryViz.csv', 'w', encoding = 'utf-8', newline = '') as f:
#     df.to_csv(f)

In [ ]:
# Memory issues. Worked in colab
# colors = ['C1', 'C3', 'C2', 'C5', 'C4', 'C0']

# ax = df.plot.bar(x = 'subset', rot=0, figsize=(12,8), width=0.7, color = colors)
# plt.savefig("/content/entity_types_10folds.png")
# plt.show()

## Processing evaluation files by annotators

### ENTITIES

In [6]:
import os
from collections import defaultdict


In [33]:
def process_anno_files (paths):
    dic = defaultdict(list)
    for path in paths: 
        with open (path, 'r', encoding = 'utf-8') as file:
            data_accuracy = pd.read_csv(file)
        data_accuracy.columns = [i for i in range(len(data_accuracy.columns))]
        for j, k, v in zip ([e for e in range(1000)], data_accuracy[0], data_accuracy[1]):
            dic['%s----%s'%(k,str(j))].append(int(v))

    df = pd.DataFrame.from_dict(dic, orient='index')
    df1 = df[[0,1,2]]
    df1['sum'] = df1.sum(axis = 1)
    df1['final_eval'] = np.where(df1['sum']>=2, 1, 0)
    df1 = df1.reset_index()
    df1['index'] = df1['index'].apply(lambda x: x.split('----')[0])
    return df1 [['index','final_eval' ]]


root_path = r'C:\Users\huyen\OneDrive - UNT System\PROJECTS\COVID19_paper\KG paper\Evaluation\stage2\ALL\entities'
filenames = os.listdir(root_path)
i=0
while i<round(len(filenames)/3):
    m=3*i
    n=m+3
    path_group = []
    for files in filenames[m:n]:
        path_group.append(root_path+'\\'+ files)
#     print (path_group)
    df = process_anno_files(path_group)
    ent_type = files.split('_')[0]
    df.columns = [ent_type, 'final-eval']
    with open(r'C:\Users\huyen\OneDrive - UNT System\PROJECTS\COVID19_paper\KG paper\Evaluation\stage2\ALL\entities_tempt\%s.csv'%ent_type, 'w', newline='', encoding = 'utf-8') as f:
        df.to_csv(f)
    i+=1

['C:\\Users\\huyen\\OneDrive - UNT System\\PROJECTS\\COVID19_paper\\KG paper\\Evaluation\\stage2\\ALL\\entities\\all_ents_sample_1000(madhu).csv', 'C:\\Users\\huyen\\OneDrive - UNT System\\PROJECTS\\COVID19_paper\\KG paper\\Evaluation\\stage2\\ALL\\entities\\all_ents_sample_1000_apps.csv', 'C:\\Users\\huyen\\OneDrive - UNT System\\PROJECTS\\COVID19_paper\\KG paper\\Evaluation\\stage2\\ALL\\entities\\all_ents_sample_1000_Mudita.csv']
['C:\\Users\\huyen\\OneDrive - UNT System\\PROJECTS\\COVID19_paper\\KG paper\\Evaluation\\stage2\\ALL\\entities\\disease_sample_1000(madhu).csv', 'C:\\Users\\huyen\\OneDrive - UNT System\\PROJECTS\\COVID19_paper\\KG paper\\Evaluation\\stage2\\ALL\\entities\\disease_sample_1000_apps.csv', 'C:\\Users\\huyen\\OneDrive - UNT System\\PROJECTS\\COVID19_paper\\KG paper\\Evaluation\\stage2\\ALL\\entities\\disease_sample_1000_Zach.csv']
['C:\\Users\\huyen\\OneDrive - UNT System\\PROJECTS\\COVID19_paper\\KG paper\\Evaluation\\stage2\\ALL\\entities\\drug_sample_1000(m

### TRIPLES

In [10]:
# processing file to get final eval by maajority voting
with open(r"C:\Users\huyen\OneDrive - UNT System\PROJECTS\COVID19_paper\KG paper\Evaluation\stage2\ALL\triples\all_triples_sample_1000_all.csv",'r', encoding ='utf-8' ) as f:
     df = pd.read_csv(f)     
df
df1 = df[['Evaluation1', 'Evaluation2', 'Evaluation3']]
df1['sum'] = df1.sum(axis = 1)
df1['final_eval'] = np.where(df1['sum']>=2, 1, 0)
df['final_eval'] = df1['final_eval']
# writing to a file
with open(r"C:\Users\huyen\OneDrive - UNT System\PROJECTS\COVID19_paper\KG paper\Evaluation\stage2\ALL\triples\all_triples_sample_1000_all_final.csv",'w', encoding ='utf-8', newline = '' ) as f:
    df.to_csv(f)

<ipython-input-10-ed6161bb6180>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['sum'] = df1.sum(axis = 1)
<ipython-input-10-ed6161bb6180>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['final_eval'] = np.where(df1['sum']>=2, 1, 0)


# ML methods to automatically label correctness of enities and triples